In [351]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler


In [434]:
sales_train = pd.read_csv('BigW_data/Sales by customer location train.csv')
sales_test = pd.read_csv('BigW_data/Sales by customer location test.csv')
# combined_df = pd.concat([Sales_train, Sales_test], ignore_index=True)

store = pd.read_csv('BigW_data/Sales by store location.csv')
media = pd.read_csv('BigW_data/Media Investment.csv')
# aus = gpd.read_file('SA4_2021_AUST_SHP_GDA2020/SA4_2021_AUST_GDA2020.shp')

### Data Cleaning 

In [353]:
sales_train.isnull().sum()

store_id                             0
sales_channel                        0
customer_state                       0
customer_postcode                    7
price_lifestage_segment          13565
customer_count                       0
transaction_count                    0
total_sale_value                     0
total_sale_value_ex_gst              0
total_promotional_sales_value        0
financial_week_end_date              0
dtype: int64

In [354]:
sales_test.isnull().sum()

store_id                            0
sales_channel                       0
customer_state                      0
customer_postcode                   1
price_lifestage_segment          2645
customer_count                      0
transaction_count                   0
total_sale_value                    0
total_sale_value_ex_gst             0
total_promotional_sales_value       0
financial_week_end_date             0
dtype: int64

In [355]:
sales_train1 = sales_train.dropna()
sales_train1 = sales_train1.reset_index(drop=True)

sales_test1 = sales_test.dropna()
sales_test1 = sales_test1.reset_index(drop=True)


In [356]:
# turn negative values in promotion sales to 0 
sales_train1.loc[sales_train1['total_promotional_sales_value'] < 0, 'total_promotional_sales_value']=0
sales_test1.loc[sales_test1['total_promotional_sales_value'] < 0, 'total_promotional_sales_value']=0


In [357]:
sales_train1.isnull().sum()

store_id                         0
sales_channel                    0
customer_state                   0
customer_postcode                0
price_lifestage_segment          0
customer_count                   0
transaction_count                0
total_sale_value                 0
total_sale_value_ex_gst          0
total_promotional_sales_value    0
financial_week_end_date          0
dtype: int64

In [358]:
store1 = store.copy()
store1['co_location_flag'] = store1['co_location_flag'].fillna(True)
store1.iloc[0,7] = '>5 KM'
store1.iloc[0,8] = '>5 KM'

In [359]:
store1.isnull().sum()

store_id                         0
store_state                      0
store_postcode                   0
store_latitude                   0
store_longitude                  0
co_location_flag                 0
sales_channel                    0
distance_to_kmart                0
distance_to_target               0
total_sale_value                 0
total_sale_value_ex_gst          0
total_promotional_sales_value    0
dtype: int64

### Combine Customer Location & Store

In [360]:
sales_train1['price_lifestage_segment'] = sales_train1['price_lifestage_segment'].astype(str)

In [361]:
sales_train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1814020 entries, 0 to 1814019
Data columns (total 11 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   store_id                       object 
 1   sales_channel                  object 
 2   customer_state                 object 
 3   customer_postcode              float64
 4   price_lifestage_segment        object 
 5   customer_count                 int64  
 6   transaction_count              int64  
 7   total_sale_value               float64
 8   total_sale_value_ex_gst        float64
 9   total_promotional_sales_value  float64
 10  financial_week_end_date        object 
dtypes: float64(4), int64(2), object(5)
memory usage: 152.2+ MB


In [362]:
store_ids_set1 = set(sales_train1['store_id'])
store_ids_set2 = set(store1['store_id'])

# Find unmatched store IDs
unmatched_ids_df1 = store_ids_set1 - store_ids_set2
unmatched_ids_df2 = store_ids_set2 - store_ids_set1

# Display unmatched store IDs
print("Unmatched Store IDs in DataFrame 1:", unmatched_ids_df1)
print("Unmatched Store IDs in DataFrame 2:", unmatched_ids_df2)


Unmatched Store IDs in DataFrame 1: set()
Unmatched Store IDs in DataFrame 2: {'f639b3bffb0910cb1de42fee016df58d'}


In [363]:
# sales train drop nan 后比store dataset 少一个store id, 所以把这个store id 从store中删除了

store1 = store1[store1['store_id'] != 'f639b3bffb0910cb1de42fee016df58d']

In [365]:
# Store dataset group by store id

unique_store = store1.groupby(['store_id']).agg({
    'store_state':'first','store_postcode':'first', 'store_latitude':'first' ,'store_longitude':'first', 
       'co_location_flag':'first', 'distance_to_kmart':'first', 'distance_to_target':'first',
    'total_sale_value': 'sum','total_promotional_sales_value':'sum','total_sale_value_ex_gst':'sum'
}).reset_index()

unique_store


,store_id,store_state,store_postcode,store_latitude,store_longitude,co_location_flag,distance_to_kmart,distance_to_target,total_sale_value,total_promotional_sales_value,total_sale_value_ex_gst
0,00f80c0962fb37ea8fb72accc75546c8,TAS,7050,-42.9759,147.3085,True,>5 KM,>5 KM,63516938.31,7741346.78,57946972.60
1,00fdea78e677259a3e44868468f24bbf,QLD,4053,-27.4093,152.9800,True,1-3 KM,Same Centre,52957737.59,7563628.70,48297404.90
2,03d9b714d7ac602a9755a662c1d65be4,QLD,4187,-19.3174,146.7285,True,3-5 KM,>5 KM,76112737.87,7932218.02,69489762.91
3,059fffb74fa5f6ebe37bd1df479c02f7,QLD,4720,-23.5169,148.2108,True,>5 KM,>5 KM,55341416.14,6735050.98,50473985.98
4,05c3e372f03989d8e904372a2767f158,SA,5158,-35.0787,138.5169,True,>5 KM,3-5 KM,30462616.26,5037218.08,27829177.50
...,...,...,...,...,...,...,...,...,...,...,...
172,fbfaede8d00839631393db8bdecbb3db,SA,5046,-35.0138,138.5447,True,Same Centre,Same Centre,73475153.85,9017465.16,67049965.38
173,fc25b8de93fc78247ad51123fd96c07d,SA,5125,-34.7753,138.7160,True,3-5 KM,3-5 KM,42522051.91,6505523.65,38834898.46
174,fc2751520dda99c0c7f267904fb483e7,NSW,2594,-34.3036,148.2702,False,>5 KM,>5 KM,42959298.99,5465555.31,39231426.97
175,fec8038ae4436caa7188b2e3f0bd216f,WA,6155,-32.0388,115.8889,True,3-5 KM,3-5 KM,51649026.67,6971991.23,47150907.22


In [366]:
# merge training set and unique store dataset 

sales_train1 = sales_train1.merge(unique_store, on='store_id', how='left')
sales_train1

,store_id,sales_channel,customer_state,customer_postcode,price_lifestage_segment,customer_count,transaction_count,total_sale_value_x,total_sale_value_ex_gst_x,total_promotional_sales_value_x,financial_week_end_date,store_state,store_postcode,store_latitude,store_longitude,co_location_flag,distance_to_kmart,distance_to_target,total_sale_value_y,total_promotional_sales_value_y,total_sale_value_ex_gst_y
0,536fcc1021bff60edb212ca9b2480757,Store,NSW,2550.0,Budget-New Families,51,77,6691.29,6146.98,1545.78,2023-03-26,NSW,2550,-36.6785,149.8398,True,>5 KM,>5 KM,38132622.27,5879501.90,34759019.81
1,151e50feaf21b61cf23e0b944abebe28,Store,VIC,3222.0,Budget-Retirees,8,8,364.53,331.40,89.72,2022-01-02,VIC,3220,-38.1467,144.3587,True,1-3 KM,Same Centre,57694483.90,7166089.98,52597105.02
2,b94d71e6bc4cc21ffbaaef5e4aca6f03,Store,QLD,4101.0,Mainstream-Young Singles/Couples,10,11,446.25,405.69,9.75,2023-03-19,QLD,4122,-27.5600,153.0811,True,Same Centre,Same Centre,92482637.90,9571752.04,84342899.73
3,336e7e53cf0e3fbc9eb41bfaf6dc4847,Store,SA,5253.0,Mainstream-Young Families,79,119,7479.89,6817.90,1365.50,2023-01-29,SA,5253,-35.1183,139.2747,True,>5 KM,>5 KM,42636413.34,6131102.71,38922466.04
4,97012ce84c33e28732cb5f12bb92363c,Store,VIC,3064.0,Premium-New Families,58,84,6172.44,5630.27,508.59,2021-11-07,VIC,3064,-37.5939,144.9103,True,Same Centre,>5 KM,54545504.41,7088165.04,49733005.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814015,17b4adf18100c8673e3f2c72dff87722,Store,QLD,4220.0,Mainstream-Midage Singles/Couples,8,9,710.79,646.18,319.40,2021-07-18,QLD,4226,-28.0805,153.3836,True,Same Centre,Same Centre,90359604.31,9031217.55,82398417.91
1814016,b16365302aa017bfadf5b34a84a84e45,Store,WA,6066.0,Mainstream-Retirees,42,46,2064.05,1886.07,268.02,2022-12-18,WA,6061,-31.8720,115.8592,True,Same Centre,3-5 KM,60844174.39,7251128.76,55488379.89
1814017,d0f1c52a0107bd8707444bbf130a0234,Store,WA,6170.0,Mainstream-Midage Singles/Couples,29,37,1795.17,1634.68,170.20,2022-02-13,WA,6966,-32.2476,115.8123,True,>5 KM,>5 KM,59498612.01,6925746.81,54288917.48
1814018,408b7c8ad06e4d9954fa2d948a01f508,Store,NSW,2769.0,Mainstream-Older Singles/Couples,32,38,2141.80,1973.61,204.08,2022-12-18,NSW,2155,-33.6917,150.9247,True,Same Centre,>5 KM,92531636.15,10145143.98,84402708.86


### Feature Engineering 

In [367]:
def extract_segment1(value):
    if value:
        if "new families" in value.lower():
            return "New Families"
        elif "retirees" in value.lower():
            return "Retirees"
        elif "older singles/couples" in value.lower():
            return "Older Singles/Couples"
        elif "young singles/couples" in value.lower():
            return "Young Singles/Couples"
        elif "young families" in value.lower():
            return "Young Families"
        elif "older families" in value.lower():
            return "Older Families"
        elif "midage singles/couples" in value.lower():
            return "Midage Singles/Couples"
    return None

sales_train1['segment'] = sales_train1['price_lifestage_segment'].apply(extract_segment1)
sales_test1['segment'] = sales_test1['price_lifestage_segment'].apply(extract_segment1)

In [368]:
def extract_segment(value):
    if value:
        if "budget" in value.lower():
            return "Budget"
        elif "premium" in value.lower():
            return "Premium"
        elif "mainstream" in value.lower():
            return "Mainstream"
    return None

sales_train1['main_segment'] = sales_train1['price_lifestage_segment'].apply(extract_segment)
sales_test1['main_segment'] = sales_test1['price_lifestage_segment'].apply(extract_segment)


In [369]:
# Distance to Target and Kmart 

def set_distance_kmart(distance):
    if distance == '1-3 KM':
        return 0.87
    elif distance == '3-5 KM':
        return 0.79
    elif distance == '<1 KM':
        return 0.95
    elif distance == '>5 KM':
        return 1
    elif distance == 'Same Centre':
        return 0.66
    else:
        return None  # Set to None for values that don't match the conditions
    
def set_distance_target(distance):
    if distance == '1-3 KM':
        return 0.96
    elif distance == '3-5 KM':
        return 0.77
    elif distance == '<1 KM':
        return 1
    elif distance == '>5 KM':
        return 0.94
    elif distance == 'Same Centre':
        return 0.73
    else:
        return None

# Apply the custom function to the 'distance_km' column
sales_train1['distance_kmart'] = sales_train1['distance_to_kmart'].apply(set_distance_kmart)
sales_train1['distance_target'] = sales_train1['distance_to_target'].apply(set_distance_target)

sales_train1['distance_kmart_target']=sales_train1['distance_kmart']*sales_train1['distance_target']


In [370]:
# co_location_flag

def set_values1(co):
    if co == True:
        return 0.68
    elif co == False:
        return 1
    else:
        return None  # Set to None for values that don't match the conditions
sales_train1['co'] = sales_train1['co_location_flag'].apply(set_values1)
# sales_train1['co'] = sales_train1['co_location_flag'].apply(set_values1)


In [371]:
# price_lifestage_segment

def set_lifestage_segments(segment):
    if segment == 'Budget-Midage Singles/Couples':
        return 0.44
    elif segment == 'Budget-New Families':
        return 0.75
    elif segment == 'Budget-Older Families':
        return 0.62
    elif segment == 'Budget-Older Singles/Couples':
        return 0.62
    elif segment == 'Budget-Retirees':
        return 0.43
    elif segment == 'Budget-Young Families':
        return 1
    elif segment == 'Budget-Young Singles/Couples':
        return 0.52
    
    elif segment == 'Mainstream-Midage Singles/Couples':
        return 0.37
    elif segment == 'Mainstream-New Families':
        return 0.51
    elif segment == 'Mainstream-Older Families':
        return 0.48
    elif segment == 'Mainstream-Older Singles/Couples':
        return 0.63
    elif segment == 'Mainstream-Retirees':
        return 0.53
    elif segment == 'Mainstream-Young Families':
        return 0.73
    elif segment == 'Mainstream-Young Singles/Couples':
        return 0.41
    
    elif segment == 'Premium-Midage Singles/Couples':
        return 0.27
    elif segment == 'Premium-New Families':
        return 0.34
    elif segment == 'Premium-Older Families':
        return 0.3
    elif segment == 'Premium-Older Singles/Couples':
        return 0.43
    elif segment == 'Premium-Retirees':
        return 0.38
    elif segment == 'Premium-Young Families':
        return 0.51
    elif segment == 'Premium-Young Singles/Couples':
        return 0.25
 
    else:
        return None  

    
sales_train1['price_lifestage_segment_num'] = sales_train1['price_lifestage_segment'].apply(set_lifestage_segments)




In [372]:
# main_segment

def set_main_segments(segment):
    if segment == 'Budget':
        return 1
    elif segment == 'Premium':
        return 0.56
    elif segment == 'Mainstream':
        return 0.81
    else:
        return None

sales_train1['main_segment_num'] = sales_train1['main_segment'].apply(set_main_segments)

In [373]:
# segment

def set_segments(segment):
    if segment == 'Midage Singles/Couples':
        return 0.45
    elif segment == 'New Families':
        return 0.72
    elif segment == 'Older Families':
        return 0.64
    elif segment == 'Older Singles/Couples':
        return 0.72
    elif segment == 'Retirees':
        return 0.57
    elif segment == 'Young Families':
        return 1
    elif segment == 'Young Singles/Couples':
        return 0.52
 
    else:
        return None  

    
sales_train1['segment_num'] = sales_train1['segment'].apply(set_segments)



In [374]:
# state

def set_states(state):
    if state == 'ACT':
        return 0.58
    elif state == 'NSW':
        return 1
    elif state == 'NT':
        return 0.86
    elif state == 'QLD':
        return 0.98
    elif state == 'SA':
        return 0.71
    elif state == 'TAS':
        return 0.8
    elif state == 'VIC':
        return 0.75
    elif state == 'WA':
        return 0.85
    else:
        return None
    
    
sales_train1['state_num'] = sales_train1['customer_state'].apply(set_states)

In [520]:
# Sales Channel (目前转变为数字形式不准确)

def set_channel(channel):
    if channel == 'Digital - ClickCollect':
        return 0.74
    elif channel == 'Digital - Delivery':
        return 0.42
    elif channel == 'Store':
        return 1
    else:
        return None 
    
sales_train1['channel_num'] = sales_train1['sales_channel'].apply(set_channel)

In [521]:
# segment * sales channel 

sales_train1['price_lifestage_segment_channel_num'] = sales_train1['channel_num']*sales_train1['price_lifestage_segment_num']

In [490]:

sales_train1['price_lifestage_segment_channel'] = sales_train1['channel_num'] * sales_train1['price_lifestage_segment_num']


In [377]:
# state & price_lifestate_segment

sales_train1['state_lifestage'] = sales_train1['state_num']*sales_train1['price_lifestage_segment_num']

In [378]:

sales_train1['promotional_price_lifestage_segment'] = sales_train1['price_lifestage_segment_num']*sales_train1['total_promotional_sales_value_x']
# sales_train1['promotional_segment'] = sales_train1['segment_num']*sales_train1['total_promotional_sales_value_x']
# sales_train1['promotional_main_segment'] = sales_train1['main_segment_num']*sales_train1['total_promotional_sales_value_x']

sales_train1['state_promotional'] = sales_train1['state_num']*sales_train1['total_promotional_sales_value_x']


In [409]:
sales_train1.corr()

/var/folders/sm/pvsh_08j0q9fp5m216cy4qhw0000gn/T/ipykernel_90736/2693956338.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  sales_train1.corr()


,customer_postcode,customer_count,transaction_count,total_sale_value_x,total_sale_value_ex_gst_x,total_promotional_sales_value_x,store_postcode,store_latitude,store_longitude,co_location_flag,total_sale_value_y,total_promotional_sales_value_y,total_sale_value_ex_gst_y,distance_kmart,distance_target,distance_kmart_target,co,price_lifestage_segment_num,main_segment_num,segment_num,state_num,channel_num,promotional_sales_channel,state_lifestage,promotional_price_lifestage_segment,state_promotional,price_lifestage_segment_channel
customer_postcode,1.000000,0.003265,0.002995,-0.007186,-0.007140,-0.003394,0.992568,0.035851,-0.601737,0.025506,0.000737,0.133101,0.001165,-0.035632,-0.126581,-0.074469,-0.025506,0.024153,0.037849,-0.001424,-0.314666,0.038534,-0.003306,-0.098348,0.000047,-0.026732,0.029841
customer_count,0.003265,1.000000,0.990485,0.928959,0.928534,0.815632,0.005137,0.056973,0.011630,-0.031558,0.013863,0.010713,0.014041,0.076600,0.052075,0.077139,0.031558,0.129498,0.079213,0.086688,0.068888,0.057657,0.815609,0.145847,0.741342,0.811378,0.141646
transaction_count,0.002995,0.990485,1.000000,0.952650,0.952302,0.848034,0.005136,0.055892,0.013749,-0.042456,0.001115,-0.001919,0.001298,0.092340,0.062551,0.093538,0.042456,0.144988,0.101418,0.091833,0.069398,0.057288,0.848009,0.160586,0.782489,0.844729,0.157423
total_sale_value_x,-0.007186,0.928959,0.952650,1.000000,0.999980,0.876994,-0.005612,0.063328,0.025366,-0.061943,0.006927,0.002121,0.007137,0.106023,0.072570,0.108627,0.061943,0.205092,0.124728,0.152067,0.082775,0.024881,0.876819,0.222313,0.842220,0.877643,0.212740
total_sale_value_ex_gst_x,-0.007140,0.928534,0.952302,0.999980,1.000000,0.876758,-0.005567,0.063244,0.025401,-0.061910,0.006916,0.002104,0.007127,0.105881,0.072486,0.108489,0.061910,0.204941,0.124849,0.151764,0.082721,0.025061,0.876585,0.222158,0.841985,0.877431,0.212620
total_promotional_sales_value_x,-0.003394,0.815632,0.848034,0.876994,0.876758,1.000000,-0.001657,0.038754,0.015771,-0.054790,-0.016163,-0.012124,-0.016006,0.091757,0.058528,0.092056,0.054790,0.208980,0.172470,0.122969,0.055606,0.066257,0.999993,0.214415,0.958913,0.991474,0.224303
store_postcode,0.992568,0.005137,0.005136,-0.005612,-0.005567,-0.001657,1.000000,0.039568,-0.606207,0.019492,-0.009776,0.125918,-0.009323,-0.029511,-0.122800,-0.067970,-0.019492,0.022900,0.036931,-0.002128,-0.305052,0.040055,-0.001567,-0.095701,0.001222,-0.024678,0.028919
store_latitude,0.035851,0.056973,0.055892,0.063328,0.063244,0.038754,0.039568,1.000000,0.080629,0.044436,0.045026,-0.019261,0.045231,0.053410,-0.047991,0.015657,-0.044436,0.009123,0.028142,-0.009655,0.500420,0.020747,0.038769,0.193666,0.031924,0.069795,0.012358
store_longitude,-0.601737,0.011630,0.013749,0.025366,0.025401,0.015771,-0.606207,0.080629,1.000000,-0.068312,-0.000732,-0.109883,-0.000426,0.078551,-0.036922,0.038201,0.068312,0.005064,0.014181,-0.004655,0.405523,-0.031339,0.015679,0.156817,0.014062,0.044012,0.000213
co_location_flag,0.025506,-0.031558,-0.042456,-0.061943,-0.061910,-0.054790,0.019492,0.044436,-0.068312,1.000000,0.218405,0.232711,0.218072,-0.196509,-0.221289,-0.246373,-1.000000,-0.022660,-0.037983,0.002191,-0.019030,0.009745,-0.054748,-0.027232,-0.045159,-0.055091,-0.021345


In [523]:
# predictor variables that can be used in OLS without causing multicollinearity
# promotional_total_sals_value/total_sale_value, customer/transaction count 
# !! It's better to obtain information about the relevant socialeconomic index if we want to include customer segment variable in our modles 



import statsmodels.api as sm

X = sales_train1[['price_lifestage_segment_channel_num','promotional_price_lifestage_segment','transaction_count', 'state_num','distance_kmart', 'distance_target', 'co']]
# price_lifestage_segment_channel_num'

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

X = sm.add_constant(scaled_X)

# Fit the OLS regression model
model = sm.OLS(sales_train1['total_sale_value_x'], X).fit()

# Get a summary of the regression results
summary = model.summary()

summary





<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_sale_value_x   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                 3.599e+06
Date:                Tue, 03 Oct 2023   Prob (F-statistic):               0.00
Time:                        14:42:03   Log-Likelihood:            -1.4533e+07
No. Observations:             1814020   AIC:                         2.907e+07
Df Residuals:                 1814012   BIC:                         2.907e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1684.8965      0.542   3110.079      0.000    1683.835    1685.958
x1            44.6839      0.579     77.240      0.000      43.550      45.818
x2           680.3673      0.917    741.807      0.000     678.570     682.165
x3          2133.3259      0.883   2414.846      0.000    2131.594    2135.057
x4            52.3689      0.543     96.397      0.000      51.304      53.434
x5            32.5368      0.607     53.575      0.000      31.346      33.727
x6            17.8285      0.609     29.255      0.000      16.634      19.023
x7            40.8185      0.560     72.950      0.000      39.722      41.915
==============================================================================
Omnibus:                  1207471.652   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       1261606383.397
Skew:                           1.778   Prob(JB):                         0.00
Kurtosis:                     132.146   Cond. No.                         3.15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [487]:
pd.set_option('display.max_columns', None)

sales_train1.head(
)

# pd.reset_option('display.max_columns')


,store_id,sales_channel,customer_state,customer_postcode,price_lifestage_segment,customer_count,transaction_count,total_sale_value_x,total_sale_value_ex_gst_x,total_promotional_sales_value_x,financial_week_end_date,store_state,store_postcode,store_latitude,store_longitude,co_location_flag,distance_to_kmart,distance_to_target,total_sale_value_y,total_promotional_sales_value_y,total_sale_value_ex_gst_y,segment,main_segment,distance_kmart,distance_target,distance_kmart_target,co,price_lifestage_segment_num,main_segment_num,segment_num,state_num,channel_num,promotional_sales_channel,state_lifestage,promotional_price_lifestage_segment,state_promotional,price_lifestage_segment_channel,price_lifestage_segment_channel_num
0,536fcc1021bff60edb212ca9b2480757,Store,NSW,2550.0,Budget-New Families,51,77,6691.29,6146.98,1545.78,2023-03-26,NSW,2550,-36.6785,149.8398,True,>5 KM,>5 KM,38132622.27,5879501.90,34759019.81,New Families,Budget,1.00,0.94,0.9400,0.68,0.75,1.00,0.72,1.00,1.0,1545.78,0.7500,1159.3350,1545.7800,0.75,0.75
1,151e50feaf21b61cf23e0b944abebe28,Store,VIC,3222.0,Budget-Retirees,8,8,364.53,331.40,89.72,2022-01-02,VIC,3220,-38.1467,144.3587,True,1-3 KM,Same Centre,57694483.90,7166089.98,52597105.02,Retirees,Budget,0.87,0.73,0.6351,0.68,0.43,1.00,0.57,0.75,1.0,89.72,0.3225,38.5796,67.2900,0.43,0.43
2,b94d71e6bc4cc21ffbaaef5e4aca6f03,Store,QLD,4101.0,Mainstream-Young Singles/Couples,10,11,446.25,405.69,9.75,2023-03-19,QLD,4122,-27.5600,153.0811,True,Same Centre,Same Centre,92482637.90,9571752.04,84342899.73,Young Singles/Couples,Mainstream,0.66,0.73,0.4818,0.68,0.41,0.81,0.52,0.98,1.0,9.75,0.4018,3.9975,9.5550,0.41,0.41
3,336e7e53cf0e3fbc9eb41bfaf6dc4847,Store,SA,5253.0,Mainstream-Young Families,79,119,7479.89,6817.90,1365.50,2023-01-29,SA,5253,-35.1183,139.2747,True,>5 KM,>5 KM,42636413.34,6131102.71,38922466.04,Young Families,Mainstream,1.00,0.94,0.9400,0.68,0.73,0.81,1.00,0.71,1.0,1365.50,0.5183,996.8150,969.5050,0.73,0.73
4,97012ce84c33e28732cb5f12bb92363c,Store,VIC,3064.0,Premium-New Families,58,84,6172.44,5630.27,508.59,2021-11-07,VIC,3064,-37.5939,144.9103,True,Same Centre,>5 KM,54545504.41,7088165.04,49733005.77,New Families,Premium,0.66,0.94,0.6204,0.68,0.34,0.56,0.72,0.75,1.0,508.59,0.2550,172.9206,381.4425,0.34,0.34


### Group by each categorical variable and use proportion of mean total sales value to turn categorical variables to numerical 


In [500]:

sales_train1.groupby(['main_segment'])['total_sale_value_x'].mean()

main_segment
Budget        2062.263988
Mainstream    1668.855332
Premium       1149.223072
Name: total_sale_value_x, dtype: float64

In [383]:
sales_train1.groupby(['customer_state'])['total_sale_value_x'].mean()

customer_state
ACT    1106.425216
NSW    1905.273026
NT     1641.390572
QLD    1835.511444
SA     1351.142865
TAS    1484.289173
VIC    1421.938253
WA     1623.004088
Name: total_sale_value_x, dtype: float64

In [384]:
sales_train1.groupby(['price_lifestage_segment'])['total_sale_value_x'].mean() / 3036.72

price_lifestage_segment
Budget-Midage Singles/Couples        0.439158
Budget-New Families                  0.747243
Budget-Older Families                0.617496
Budget-Older Singles/Couples         0.622212
Budget-Retirees                      0.432978
Budget-Young Families                1.000000
Budget-Young Singles/Couples         0.517195
Mainstream-Midage Singles/Couples    0.370202
Mainstream-New Families              0.512663
Mainstream-Older Families            0.479408
Mainstream-Older Singles/Couples     0.630931
Mainstream-Retirees                  0.530621
Mainstream-Young Families            0.731999
Mainstream-Young Singles/Couples     0.410341
Premium-Midage Singles/Couples       0.269670
Premium-New Families                 0.341299
Premium-Older Families               0.303588
Premium-Older Singles/Couples        0.433644
Premium-Retirees                     0.382659
Premium-Young Families               0.505396
Premium-Young Singles/Couples        0.250778
Name: tota

In [385]:
sales_train1.groupby(['distance_to_target'])['total_sale_value_x'].mean() 

distance_to_target
1-3 KM         1884.439808
3-5 KM         1506.608861
<1 KM          1957.985187
>5 KM          1847.794114
Same Centre    1432.693221
Name: total_sale_value_x, dtype: float64

In [386]:
sales_train1.groupby(['co_location_flag'])['total_sale_value_x'].mean() /4043.492276

co_location_flag
False    0.59381
True     0.40619
Name: total_sale_value_x, dtype: float64

In [387]:
1884.439808+1506.608861+1957.985187+1847.794114+1432.693221

8629.521191

In [418]:
sales_train1.groupby(['price_lifestage_segment'])['total_promotional_sales_value_x'].mean()

price_lifestage_segment
Budget-Midage Singles/Couples        237.182902
Budget-New Families                  378.299630
Budget-Older Families                348.501292
Budget-Older Singles/Couples         369.060261
Budget-Retirees                      238.036095
Budget-Young Families                514.169683
Budget-Young Singles/Couples         296.353009
Mainstream-Midage Singles/Couples    156.835405
Mainstream-New Families              203.074366
Mainstream-Older Families            202.405025
Mainstream-Older Singles/Couples     273.881555
Mainstream-Retirees                  216.308903
Mainstream-Young Families            283.849949
Mainstream-Young Singles/Couples     175.207371
Premium-Midage Singles/Couples       109.565409
Premium-New Families                 125.694982
Premium-Older Families               119.749583
Premium-Older Singles/Couples        177.190857
Premium-Retirees                     145.986552
Premium-Young Families               183.253268
Premium-Young Si

In [494]:
sales_train1.drop('promotional_sales_channel',axis=1)

,store_id,sales_channel,customer_state,customer_postcode,price_lifestage_segment,customer_count,transaction_count,total_sale_value_x,total_sale_value_ex_gst_x,total_promotional_sales_value_x,financial_week_end_date,store_state,store_postcode,store_latitude,store_longitude,co_location_flag,distance_to_kmart,distance_to_target,total_sale_value_y,total_promotional_sales_value_y,total_sale_value_ex_gst_y,segment,main_segment,distance_kmart,distance_target,distance_kmart_target,co,price_lifestage_segment_num,main_segment_num,segment_num,state_num,channel_num,state_lifestage,promotional_price_lifestage_segment,state_promotional,price_lifestage_segment_channel,price_lifestage_segment_channel_num
0,536fcc1021bff60edb212ca9b2480757,Store,NSW,2550.0,Budget-New Families,51,77,6691.29,6146.98,1545.78,2023-03-26,NSW,2550,-36.6785,149.8398,True,>5 KM,>5 KM,38132622.27,5879501.90,34759019.81,New Families,Budget,1.00,0.94,0.9400,0.68,0.75,1.00,0.72,1.00,1.0,0.7500,1159.3350,1545.7800,0.75,0.75
1,151e50feaf21b61cf23e0b944abebe28,Store,VIC,3222.0,Budget-Retirees,8,8,364.53,331.40,89.72,2022-01-02,VIC,3220,-38.1467,144.3587,True,1-3 KM,Same Centre,57694483.90,7166089.98,52597105.02,Retirees,Budget,0.87,0.73,0.6351,0.68,0.43,1.00,0.57,0.75,1.0,0.3225,38.5796,67.2900,0.43,0.43
2,b94d71e6bc4cc21ffbaaef5e4aca6f03,Store,QLD,4101.0,Mainstream-Young Singles/Couples,10,11,446.25,405.69,9.75,2023-03-19,QLD,4122,-27.5600,153.0811,True,Same Centre,Same Centre,92482637.90,9571752.04,84342899.73,Young Singles/Couples,Mainstream,0.66,0.73,0.4818,0.68,0.41,0.81,0.52,0.98,1.0,0.4018,3.9975,9.5550,0.41,0.41
3,336e7e53cf0e3fbc9eb41bfaf6dc4847,Store,SA,5253.0,Mainstream-Young Families,79,119,7479.89,6817.90,1365.50,2023-01-29,SA,5253,-35.1183,139.2747,True,>5 KM,>5 KM,42636413.34,6131102.71,38922466.04,Young Families,Mainstream,1.00,0.94,0.9400,0.68,0.73,0.81,1.00,0.71,1.0,0.5183,996.8150,969.5050,0.73,0.73
4,97012ce84c33e28732cb5f12bb92363c,Store,VIC,3064.0,Premium-New Families,58,84,6172.44,5630.27,508.59,2021-11-07,VIC,3064,-37.5939,144.9103,True,Same Centre,>5 KM,54545504.41,7088165.04,49733005.77,New Families,Premium,0.66,0.94,0.6204,0.68,0.34,0.56,0.72,0.75,1.0,0.2550,172.9206,381.4425,0.34,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814015,17b4adf18100c8673e3f2c72dff87722,Store,QLD,4220.0,Mainstream-Midage Singles/Couples,8,9,710.79,646.18,319.40,2021-07-18,QLD,4226,-28.0805,153.3836,True,Same Centre,Same Centre,90359604.31,9031217.55,82398417.91,Midage Singles/Couples,Mainstream,0.66,0.73,0.4818,0.68,0.37,0.81,0.45,0.98,1.0,0.3626,118.1780,313.0120,0.37,0.37
1814016,b16365302aa017bfadf5b34a84a84e45,Store,WA,6066.0,Mainstream-Retirees,42,46,2064.05,1886.07,268.02,2022-12-18,WA,6061,-31.8720,115.8592,True,Same Centre,3-5 KM,60844174.39,7251128.76,55488379.89,Retirees,Mainstream,0.66,0.77,0.5082,0.68,0.53,0.81,0.57,0.85,1.0,0.4505,142.0506,227.8170,0.53,0.53
1814017,d0f1c52a0107bd8707444bbf130a0234,Store,WA,6170.0,Mainstream-Midage Singles/Couples,29,37,1795.17,1634.68,170.20,2022-02-13,WA,6966,-32.2476,115.8123,True,>5 KM,>5 KM,59498612.01,6925746.81,54288917.48,Midage Singles/Couples,Mainstream,1.00,0.94,0.9400,0.68,0.37,0.81,0.45,0.85,1.0,0.3145,62.9740,144.6700,0.37,0.37
1814018,408b7c8ad06e4d9954fa2d948a01f508,Store,NSW,2769.0,Mainstream-Older Singles/Couples,32,38,2141.80,1973.61,204.08,2022-12-18,NSW,2155,-33.6917,150.9247,True,Same Centre,>5 KM,92531636.15,10145143.98,84402708.86,Older Singles/Couples,Mainstream,0.66,0.94,0.6204,0.68,0.63,0.81,0.72,1.00,1.0,0.6300,128.5704,204.0800,0.63,0.63


In [514]:
sales_train1.groupby(['sales_channel'])['total_sale_value_x'].mean() /1694.856398

sales_channel
Digital - ClickCollect    0.738631
Digital - Delivery        0.420635
Store                     1.000000
Name: total_sale_value_x, dtype: float64